## Model Development Using Scikit-Learn
- Please open this notebook in edit mode.

#### Load enhanced customer history data as pandas dataframe from the csv
- This is the csv file we stored as asset in the previous step.

In [1]:
import pandas as pd
from project_lib import Project

project = Project.access()
enhanced_customer_churn=pd.read_csv(project.get_file("enhanced_customer_history_pandas.csv")).drop(columns=['ID'])
enhanced_customer_churn['CHURN'] = enhanced_customer_churn['CHURN'].map({'T': 1, 'F': 0})
enhanced_customer_churn.head()

LONGDISTANCE  INTERNATIONAL  LOCAL  DROPPED PAYMETHOD LOCALBILLTYPE  \
0            23              0    206        0        CC        Budget   
1            29              0     45        0        CH     FreeLocal   
2            24              0     22        0        CC     FreeLocal   
3            26              0     32        1        CC        Budget   
4            12              0     46        4        CC     FreeLocal   

  LONGDISTANCEBILLTYPE  USAGE  RATEPLAN  CHURN GENDER STATUS  CHILDREN  \
0       Intnl_discount    229         3      1      F      S         1   
1             Standard     75         2      0      M      M         2   
2             Standard     47         3      0      M      M         0   
3             Standard     59         1      0      M      S         2   
4             Standard     58         1      0      M      M         2   

   ESTINCOME CAROWNER        AGE  Phase  
0   38000.00        N  24.393333  Adult  
1   29616.00        N  49.426667  Adult  
2   19732.80        N  50.673333  Adult  
3      96.33        N  56.473333  Adult  
4   53010.80        N  18.840000  Adult

### Data preprocessing
- Here we are showing a few data preprocessing steps.

#### Select categorical columns

In [2]:
cat_cols = ["GENDER", "STATUS", "CAROWNER", "PAYMETHOD", "LOCALBILLTYPE", "LONGDISTANCEBILLTYPE","Phase"]

#### Select numerical columns

In [3]:
num_cols = ["CHILDREN", "ESTINCOME", "AGE", "LONGDISTANCE", "INTERNATIONAL", "LOCAL","DROPPED","USAGE","RATEPLAN"]

#### Import required python modules

In [4]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer

#### Let's create a preprocessing step
- It does transformation in categorial columns and normalization in numerial columns

In [5]:
ohenc = OneHotEncoder(handle_unknown='ignore')
scaler = StandardScaler() 
preprocessor = ColumnTransformer(transformers=[
    ('num', scaler, num_cols),
    ('cat', ohenc, cat_cols)
])

### Prepare training data and pipeline 
- Pipeline includes preprocessing steps and a tree based classifier.

In [6]:
y = enhanced_customer_churn["CHURN"]
X = enhanced_customer_churn[num_cols+cat_cols]
pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('modeling', RandomForestClassifier())])  

### Let's evaluate our pipeline with K-fold (K=10) cross validation

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(pipeline, X, y, cv=10,scoring="f1_micro")
scores.mean()

### Fit the pipeline with data
- It generates a trained model, which we will store in the Watson ML repository.

In [ ]:
#using 70% to fit the model
df=enhanced_customer_churn.sample(frac=0.7);

model=pipeline.fit(df[num_cols+cat_cols],df["CHURN"])

## Store trained model
- We will store the trained model in the Watson Machine Learning (WML) repository.
- Before storing a trained model, we will create a deploy space if it does not exist already.


##### Create a WML client
- It fetches required credentials from the environment. Please don't replace anything.

In [9]:
import os
from watson_machine_learning_client import WatsonMachineLearningAPIClient
token = os.environ['USER_ACCESS_TOKEN']
wml_credentials = {
   "token": token,
   "instance_id" : "wml_local",
   "url": os.environ['RUNTIME_ENV_APSX_URL'],
   "version": "2.5.0"
}
wml_client = WatsonMachineLearningAPIClient(wml_credentials)

##### Lets create a deployment space 
- In this tutorial, we are using `Customer_Churn_Deployment_Space` as the default deployment space.
- The following cell creates a deployment space with the name above if it does not exist already.
- We need a deployment space to store a model in a later part of this notebook.

In [ ]:
DEFAULT_DEPLOYMENT_SPACE="Customer_Churn_Deployment_Space"

filtered_spaces=[ space['metadata']['guid'] for space in wml_client.spaces.get_details()['resources'] if space['entity']['name']== DEFAULT_DEPLOYMENT_SPACE]
if len(filtered_spaces)>0:
    space_id=filtered_spaces[0]
else:
    space_metadata = {
        wml_client.spaces.ConfigurationMetaNames.NAME: DEFAULT_DEPLOYMENT_SPACE,
        wml_client.spaces.ConfigurationMetaNames.DESCRIPTION: DEFAULT_DEPLOYMENT_SPACE,
    }
    space_details=wml_client.spaces.store(meta_props=space_metadata);
    space_id=space_details['metadata']['guid']
    
print(space_id)


### Prepare metadata for storing the model
- <font color='red'>Please provide a unique model name to `MODEL_NAME` before running the following cell.</font>
- Note: `RUNTIME_UID` is very important for metadata.


In [94]:
MODEL_NAME="<model name>"
project = Project.access()
project_id=project.get_metadata()['metadata']['guid']

metadata = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME,
    wml_client.repository.ModelMetaNames.RUNTIME_UID: "scikit-learn_0.20-py3.6",
    wml_client.repository.ModelMetaNames.TYPE: "scikit-learn_0.20"
}


### Store model in the WML repository
- It gets schema details from the training data and target.

In [ ]:
#setting default space
wml_client.set.default_space(space_id)

#Storing 
model_details = wml_client.repository.store_model( model, meta_props=metadata,training_data=X, training_target=y)
model_details

### Store training data to DB2
- This step is needed for openscale. Openscale will use the training data for monitoring.

##### Show all available  DB2 connections
- It should show at least one connection we created in `Step 1 - Data Provisioning and Governance`

In [ ]:
from project_lib import Project
project = Project.access()
project.get_connections()

### Create a db2 sqlalchemy engine from the credential
- <font color="red">Pick a DB2 connection name from the list above and assign to `CONNECTION_NAME` before running the following cell.</font>

In [ ]:
CONNECTION_NAME="<connection name>"

# get connection credential
DB2Credentials = project.get_connection(name=CONNECTION_NAME)

import sqlalchemy
connection_string="ibm_db_sa://{0}:{1}@{2}:{3}/{4}".format(
    DB2Credentials['username'],
    DB2Credentials['password'],
    DB2Credentials['host'],
    DB2Credentials['port'],
    DB2Credentials['database']);

db2 = sqlalchemy.create_engine(connection_string)

## Create a database table and populate data 
- <font color="red">Please provide a unique table name to `TABLE_NAME` before running the following cell.</font> 
- Make sure the table name is written in ALL CAPS.
- It removes if a table already exists

In [20]:
TABLE_NAME='<TABLE NAME>'

# drop if table exists
from pandas.io import sql
sql.execute('DROP TABLE IF EXISTS %s'%TABLE_NAME.upper(), db2)

#write data to the db
enhanced_customer_churn.to_sql(TABLE_NAME.upper(), con=db2,index=False);

## Verify data in the database table

In [21]:
edf=pd.read_sql("select * from "+TABLE_NAME.upper(),db2)
edf.head(5)

longdistance  international  LOCAL  dropped paymethod localbilltype  \
0            17              0     39        0        CC        Budget   
1            22              0    103        1        CC     FreeLocal   
2            20              8     80        0        CH        Budget   
3            12              0      7        0      Auto        Budget   
4             8              0     22        0        CC     FreeLocal   

  longdistancebilltype  USAGE  rateplan  churn gender status  children  \
0       Intnl_discount     57         3      0      F      M         2   
1             Standard    126         4      1      F      S         0   
2       Intnl_discount    109         4      1      M      S         1   
3       Intnl_discount     20         4      1      F      S         1   
4             Standard     31         1      0      F      M         2   

   estincome carowner        age  Phase  
0    47639.1        N  42.000000  Adult  
1    39461.0        N  22.226667  Adult  
2    16432.1        Y  20.920000  Adult  
3      540.4        Y  39.213333  Adult  
4    80087.7        N  41.920000  Adult